## setting up a jupyter notebook

To author documents in jupyter lab and generate HTML output, use the following YAML as a template.  This should be located in the first cell of the jupyter document; the cell should be "raw"; and it should be tagged as "yaml".  To tag the cell, use the menu -- view / appearance / show right sidebar.  Then, on the right sidebar, under Common Tools / Cell Tags, enter "yaml" in the text field button and hit return.

```yaml
---
title: "metayer: authoring tools"
output: rmarkdown::html_vignette
vignette: >
  %\VignetteIndexEntry{metayer: authoring tools}
  %\VignetteEngine{knitr::rmarkdown}
  %\VignetteEncoding{UTF-8}
---
```

### load the metayer package

In [ ]:
suppressMessages({
  library(magrittr)
  library(metayer)
})

## outputting text

Some objects are efficiently described using simple text output.  To do this across publishing contexts and in a uniform way, use `display_text`:

In [ ]:
# the default list output
lst <- list(foo = 42)
display_text(lst)

In [ ]:
# a tibble
a <- 1:5
tbl <- tibble::tibble(a, b = a * 2, c = 1)
display_text(tbl)

## source highlighting

Source highlighting of a file is accomplished with `display_source`:

In [ ]:
display_source("./tests/testthat/fixtures/display_source/foo.r")

`display_source` can also take a directory path:

In [ ]:
display_source("./tests/testthat/fixtures/display_source")

### highlight theme

Changing the knitr theme is accomplished by providing a `theme` argument.  Refer to [http://animation.r-forge.r-project.org/knitr/](http://animation.r-forge.r-project.org/knitr/) for an exhaustive list of themes.

In [ ]:
display_source("./tests/testthat/fixtures/display_source/foo.r", theme = "pablo")

### programmatically highlight code

It's also possible to provide code directly to `display_source` by setting `raw` to TRUE:

In [ ]:
here::here("./tests/testthat/fixtures/display_source/foo.r") %>%
  xfun::file_string() %>%
  display_source(raw = TRUE)

## building vignettes

Store jupyter notebooks in the ./vignettes directory.  Then, supposing a vignette named "my_vignette.ipynb" exists, creating the corresponding vignette is simple:

```R
build_vignette( "my_vignette.ipynb" )
```